# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

<font color="paleblue">**My solution conssists of 3 rounds of scraping and here's why:**
1. movie_nr and title: these are immediately available at the starting page
2. year, runtime, rating, genre, plot, votes: these are in the overlay that needs to be triggered by a script
3. director, stars: available in the same overlay as the above, but i couldn't scrape them from there, so i return to the starting page, get url for each title, and scrape the director and stars from each title's page.

Possible improvement:
Skip the part 2. The info scraped from the part 2 script to be scraped from each title's page, as in the step 3.

**Why are there 2 separate dataframes?**
- Attempting to straightaway create a single df from all the scraped data results in the title column having only 1 value.
- Only when I separate the [director, stars] in another df, then concat it with the 1st df, then everything is ok.

In [ ]:
# Install the required package
# %pip install selenium beautifulsoup4

# Required imports
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import requests
import pandas as pd

def scrape_imdb(base_url, title_type, user_rating, start_date, end_date):

    # Initialize lists to store data
    movie_nr = []
    title = []
    year = []
    runtime = []
    rating = []
    genre = []
    plot = []
    votes = []
    directors = []
    stars = []
    gross = []

    # Set up Selenium WebDriver. If using chromedriver, the path not required if the chromedriver placed in /usr/local/bin/
    driver = webdriver.Safari()

    # Construct the URL with parameters
    url = f"{base_url}/?title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating}"

    # Load the page
    driver.get(url)

    # Wait for the page to fully load. Adjust this if needed depending on the internet speed
    time.sleep(5)

    # Extract the HTML content
    html_content = driver.page_source

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Close the browser when done
    driver.quit()

    # Parse the movie number and title
    for titles in soup.find_all("h3", class_="ipc-title__text"):
        text = titles.getText()
        if ". " in text:
            parts = text.split(". ", 1)
            movie_nr.append(parts[0])
            title.append(parts[1])

    # Parse the content of the overlay to get further info
    script_tag = soup.find("script", id="__NEXT_DATA__")
    json_data = json.loads(script_tag.string)
    title_results = json_data["props"]["pageProps"]["searchResults"]["titleResults"]["titleListItems"]


    for movie in title_results:
        year.append(movie.get("releaseYear"))
        runtime.append(int(movie.get("runtime")/60))
        rating.append(movie.get("ratingSummary")["aggregateRating"])
        genre.append(', '.join(movie.get("genres", [])))
        plot.append(movie.get("plot"))
        votes.append(movie.get("ratingSummary")["voteCount"])


    # Create the first df
    df1 = pd.DataFrame({
        "movie_nr": movie_nr,
        "title": title,
        "year": year,
        "runtime": runtime,
        "genre": genre,
        "plot": plot,
        "rating": rating,
        "votes": votes
    })

    # Fetch director, stars, gross
    for title in soup.find_all("a", class_="ipc-title-link-wrapper"):
        url = "https://www.imdb.com/" + title.get("href")
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        script_tag = soup.find("script", type="application/ld+json")
        json_data = json.loads(script_tag.string)

        directors.append(', '.join([d["name"] for d in json_data["director"]]))
        stars.append(', '.join([a["name"] for a in json_data["actor"]]))

        # Fetch the gross revenue
        gross_tag = soup.find("li", attrs={"data-testid": "title-boxoffice-cumulativeworldwidegross"})
        if gross_tag:
            gross_value = gross_tag.find("span", class_="ipc-metadata-list-item__list-content-item").get_text()
            gross.append(gross_value)
        else:
            gross.append("N/A")
        
        time.sleep(1)

    # Create the 2nd df
    df2 = pd.DataFrame({
        "director": directors,
        "stars": stars,
        "gross": gross
    })

    # Concat the 2 dataframes.
    df = pd.concat([df1, df2], axis=1)
    return df


# Give the parameters
base_url = "https://www.imdb.com/search/title"
title_type = "feature"
start_date = "1990-01-01"
end_date = "1992-12-31"
user_rating = "7.5,10"

df_films = scrape_imdb(base_url, title_type, user_rating, start_date, end_date)
df_films

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

<font color="paleblue">At the time of re-doing this lab, IMDB employs lazy loading. URL doesn't change when more results are loaded, and I was unable to find a way to do the bonus task.</font>